In [1]:
#install.packages("CausalImpact")

In [2]:
# Load libraries
library(CausalImpact)
library(ggplot2)

Loading required package: bsts
Warning message:
"package 'bsts' was built under R version 3.6.3"Loading required package: BoomSpikeSlab
Warning message:
"package 'BoomSpikeSlab' was built under R version 3.6.3"Loading required package: Boom
Warning message:
"package 'Boom' was built under R version 3.6.3"Loading required package: MASS

Attaching package: 'Boom'

The following object is masked from 'package:stats':

    rWishart


Attaching package: 'BoomSpikeSlab'

The following object is masked from 'package:stats':

    knots

Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Loading required package: xts
Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: 'bsts'

The following object is masked from 'package:BoomSpikeSlab':

    SuggestBurn

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures 

In [3]:
# Pharmaceutical Companies
pharma_companies <- list(
    "PFE" = "Pfizer Inc.",
    "JNJ" = "Johnson & Johnson",
    "NVS" = "Novartis AG",
    "ROG" = "Roche Holding AG",
    "MRK" = "Merck & Co., Inc.",
    "SNY" = "Sanofi",
    "GSK" = "GlaxoSmithKline plc",
    "AZN" = "AstraZeneca PLC",
    "LLY" = "Eli Lilly and Company",
    "ABBV" = "AbbVie Inc."
)

# Consumer Staples
consumer_staples <- list(
    "KO" = "The Coca-Cola Company (KO)",
    "CL" = "Colgate-Palmolive Company (CL)",
    "KMB" = "Kimberly-Clark Corporation (KMB)",
    "UL" = "Unilever (UL)"
)

# Retail
retail <- list(
    "AMZN" = "Amazon.com, Inc.",
    "ITX.MC" = "Industria de Diseño Textil, S.A.",
    "HNNMY" = "Hennes & Mauritz AB",
    "MC.PA" = "LVMH Moët Hennessy Louis Vuitton S.E.",
    "9983.T" = "Fast Retailing Co., Ltd."
)

# Global Economy Indicators
global_economy_indicators <- list(
    "^GSPC" = "S&P 500 Index",
    "^IXIC" = "NASDAQ Composite",
    "^FTSE" = "FTSE 100 (^FTSE)",
    "^DJI" = "Dow Jones Industrial Average",
    "MSCI" = "MSCI World Index"
)

industry_tickers <- list(
    "Pharma" = pharma_companies,
    "Consumables" = consumer_staples,
    "Retail" = retail,
    "Global Economy" = global_economy_indicators
)

tickers <- c(
    names(pharma_companies),
    names(consumer_staples),
    names(retail),
    names(global_economy_indicators)
)


In [4]:
# Calculate monthly average
cacl_monthly_avg_kpi <- function(stock_df, kpi = "Adj Close") {
    kpi_df <- stock_df[kpi]
    
    monthly_avg <- aggregate(kpi_df, by = list(format(stock_df$Date, "%Y-%m")), mean, na.rm=TRUE)
    colnames(monthly_avg) <- c("Date", kpi)
    
    return(monthly_avg)
}


In [5]:
# Pre-COVID Lockdown 
pre.period <- as.Date(c("2017-01-01", "2020-02-01"))
# COVID Lockdown
post.period <- as.Date(c("2020-03-01", "2020-12-01"))

In [6]:
raw_dir = "./data/raw/"
kpi = "Adj.Close"

dates <- seq.Date(pre.period[1], post.period[2], by = "month")
monthly_adj_close <- data.frame(Date = dates)
rownames(monthly_adj_close) <- dates
monthly_adj_close <- monthly_adj_close[, -1] 


for (ticker in tickers) {
    # Load data
    ticker_df <- read.csv(paste0(raw_dir, ticker, ".csv"), row.names = 1)

    # Convert Index to date
    row_dates <- as.Date(row.names(ticker_df))
    ticker_df <- cbind(Date = row_dates, ticker_df)

    # Calc Monthly Average
    ticker_df <- cacl_monthly_avg_kpi(ticker_df, kpi)
    colnames(ticker_df) <- c("Date", ticker)
    #Combine in a single table
    monthly_adj_close <- cbind(monthly_adj_close, ticker_df[ticker])
}


In [7]:
for (industry in list(pharma_companies, consumer_staples, retail)){
    for (company in names(industry)){
        print(paste("===== ", company, " ====="))
        impact_tickers <- c(company, names(global_economy_indicators))
        ticker_df = monthly_adj_close[impact_tickers]

        ticker_zoo = zoo(ticker_df,order.by = as.Date(row.names(ticker_df)))
        impact <- CausalImpact(ticker_zoo, pre.period, post.period)
        plot(impact) + labs(title =company)
        
        ggsave(paste0("figures/R/",company,".png"), width=15, height=5, dpi=300)

        causal_impact_table <- capture.output(summary(impact))
        writeLines(causal_impact_table, paste0("figures/R/",company,".causal_impact.txt"))

        summary_report <- capture.output(summary(impact, "report"))
        writeLines(summary_report, paste0("figures/R/",company,".summary_report.txt"))
        
    }
}

[1] "=====  PFE  ====="
[1] "=====  JNJ  ====="
[1] "=====  NVS  ====="
[1] "=====  ROG  ====="
[1] "=====  MRK  ====="
[1] "=====  SNY  ====="
[1] "=====  GSK  ====="
[1] "=====  AZN  ====="
[1] "=====  LLY  ====="
[1] "=====  ABBV  ====="
[1] "=====  AMZN  ====="
[1] "=====  ITX.MC  ====="
[1] "=====  HNNMY  ====="
[1] "=====  MC.PA  ====="
[1] "=====  9983.T  ====="


ERROR: Error in parse(text = x, keep.source = FALSE): <text>:1:6: unexpected symbol
1: 9983.T
         ^
